### CS182 project - Deliver ideas of CGCNN

In [1]:
### General Explanation

### Embedding Crystal Graph

In [2]:
# Note: Need pymatgen
# TODO:Explain about Crystal Graph

In [10]:
import os
import sys
import csv
import json
import torch
import torch.nn as nn
import random
import warnings
warnings.filterwarnings('ignore')
import functools
import numpy as np

from pymatgen.core.structure import Structure
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

from data_utils import CIFData
from data_utils import AtomCustomJSONInitializer
from data_utils import AtomInitializer
from data_utils import GaussianDistance

In [2]:
# Let's convert salt (NaCl) to crystal graph.
# You can use print method to see the lattice and position of Na, Cl
# Atoms in the cell.
nacl = Structure.from_file('hw_data/cifs/1000041.cif')
print(nacl)

Full Formula (Na4 Cl4)
Reduced Formula: NaCl
abc   :   5.620000   5.620000   5.620000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Na+   0    0    0
  1  Na+   0    0.5  0.5
  2  Na+   0.5  0    0.5
  3  Na+   0.5  0.5  0
  4  Cl-   0.5  0.5  0.5
  5  Cl-   0.5  0    0
  6  Cl-   0    0.5  0
  7  Cl-   0    0    0.5


In [3]:
# add visualization of the structure using the CIF file 

In [5]:
# First we will convert atoms to atomistic features using
# pre-difined atom embedding.
# atom_init.json is containing vector embedding of atoms,
# Where key (1, 2, 3, ..., 100) represent atomic number
# and values are embedding vectors.
# You can try different atom embedding too.

# Load embedding file.
element_embedding_file = 'hw_data/atom_init.json'
with open(element_embedding_file) as f:
    elem_embedding = json.load(f)
elem_embedding = {int(key): value for key, value
                  in elem_embedding.items()}

# Encode crsytal data to atomistic features.
# Atom_feature shd have shape of (# of atoms, len(embedding vector))
atom_fea = np.vstack([elem_embedding[nacl[i].specie.number]
                      for i in range(len(nacl))])
atom_fea = torch.Tensor(atom_fea)

assert atom_fea.shape == (8,92)

In [6]:
# Next, we will get neighbor information from each atoms in the cell.
# We will get help from pymatgen package.
# get_all_neighbor function of structure object returns atoms within
# the input radius. Note that here len(all_nbrs) is 8 since there are
# 8 atoms (4 Na+, 4 Cl-) in the cell. Each list contain the neighbor
# atom information considering periodicity.
# We will use 12 nearest neighbors after sorting with distance.

all_nbrs = nacl.get_all_neighbors(r = 8, include_index=True)
all_nbrs = [sorted(nbrs, key=lambda x: x[1]) for nbrs in all_nbrs]

assert len(all_nbrs) == 8

nbr_fea_idx, nbr_fea = [], []
for nbr in all_nbrs:
    # Note: x[1] returns distance to neighbors.
    # Note: x[2] returns index of original structure object
    nbr_fea_idx.append(list(map(lambda x: x[2],
                                nbr[:12])))
    nbr_fea.append(list(map(lambda x: x[1],
                            nbr[:12])))

# nbr_fea_idx contain information of nearest neighbor atoms 
# from ith row (ith atom in the cell)
# For example, 0th atom (Na+ (0.0000, 0.0000, 0.0000)) is neighbored
# with 5th, 6th, 7th, etc...
# nbr_fea contain information of nearest neighbot distance.
nbr_fea_idx, nbr_fea = np.array(nbr_fea_idx), np.array(nbr_fea)
nbr_fea_idx = torch.LongTensor(nbr_fea_idx)

nbr_fea_idx_standard = torch.LongTensor([[5, 6, 7, 7, 6, 5, 2, 1, 2, 1, 3, 3],
                                         [4, 7, 6, 7, 6, 4, 2, 0, 3, 2, 3, 3],
                                         [4, 7, 5, 7, 5, 4, 0, 3, 1, 3, 3, 1],
                                         [4, 6, 5, 6, 5, 4, 2, 1, 1, 2, 0, 0],
                                         [3, 1, 2, 3, 2, 1, 6, 5, 6, 5, 7, 7],
                                         [2, 3, 0, 3, 0, 2, 4, 4, 6, 6, 4, 7],
                                         [1, 3, 0, 3, 0, 1, 4, 4, 5, 5, 5, 4],
                                         [2, 1, 0, 2, 1, 0, 4, 5, 5, 4, 6, 6]])

assert torch.equal(nbr_fea_idx, nbr_fea_idx_standard)

In [7]:
# Now we have two features, atomic feature and neighbor feature.
# Note that neighbor feature is discontionous information with respect to the distanc.
# Therefore we will expand neighbor feature using Gaussian Kernel (or Gaussian filter)
# https://en.wikipedia.org/wiki/Gaussian_filter
dmin = 0
dmax = 12
step = 0.2
var = step
filter_step = np.arange(dmin, dmax+step, step)

def expand(distances):
    # Ask student to do this
    return np.exp(-(distances[..., np.newaxis] - filter_step)**2 / var**2)

nbr_fea_gaussian = expand(nbr_fea)

In [8]:
assert np.array_equal(nbr_fea_gaussian, nbr_fea)

AssertionError: 

In [9]:
gdf = GaussianDistance(dmin=0, dmax=12, step=0.2)
nbr_fea = gdf.expand(nbr_fea)

In [10]:
# Will not be used
data_dir = './cgcnn_data/sample-regression'
test = CIFData(data_dir)
(atom_fea, nbr_fea, nbr_fea_idx), target, cif_id = test[-1]

In [11]:
# Will not be used
structures, _, _ = test[-1]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

### Build a Model

In [12]:
# TODO: Graphics of layers

In [2]:
from model import CrystalGraphConvNet

In [14]:
# Simple test
# Set seed using manual_seed
# TODO explanation about crystal_atom_idx
torch.manual_seed(123)
crystal_atom_idx = [torch.Tensor([0, 1, 2, 3, 4, 5, 6, 7]).long()]
model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len)
model.forward(atom_fea, nbr_fea, nbr_fea_idx, crystal_atom_idx)

tensor([[0.6627]], grad_fn=<AddmmBackward0>)

### Training

In [2]:
from data_utils import collate_pool, get_train_val_test_loader
from train_utils import Normalizer, train, validate, save_checkpoint
from model import CrystalGraphConvNet
from random import sample

In [3]:
torch.manual_seed(123)

# set parameters
data_dir = './hw_data/perovskite_energy'
batch_size = 8
train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1

# get dataset

dataset = CIFData(data_dir)
collate_fn = collate_pool
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset=dataset,
    collate_fn=collate_fn,
    batch_size=batch_size,
    train_ratio=train_ratio,
    val_ratio=val_ratio,
    test_ratio=test_ratio,
    return_test=True)

In [4]:
# normalize target

if len(dataset) < 500:
    warnings.warn('Dataset has less than 500 data points. '
                    'Lower accuracy is expected. ')
    sample_data_list = [dataset[i] for i in range(len(dataset))]
else:
    sample_data_list = [dataset[i] for i in
                        sample(range(len(dataset)), 500)]
_, sample_target, _ = collate_pool(sample_data_list)
normalizer = Normalizer(sample_target)

/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [5]:
# build model
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

# number of hidden atom features in conv layers
atom_fea_len = 64
# number of hidden features after pooling
h_fea_len = 128
# number of conv layers
n_conv = 3
# number of hidden layers after pooling
n_h = 1

model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len,
                            atom_fea_len=atom_fea_len,
                            n_conv=n_conv,
                            h_fea_len=h_fea_len,
                            n_h=n_h)

In [6]:
# set hyperparameters
epochs = 15
criterion = nn.MSELoss()
lr = 0.01
momentum = 0.9
weight_decay = 0

optimizer = optim.SGD(model.parameters(), lr,
                              momentum=momentum,
                              weight_decay=weight_decay)

# optimizer = optim.Adam(model.parameters(), lr,
#                         weight_decay=weight_decay)
lr_milestones = [100]
scheduler = MultiStepLR(optimizer, milestones=lr_milestones,
                            gamma=0.1)

In [11]:
for epoch in range(epochs):
    best_mae_error = 1e10
    # train for one epoch
    # TO-Do, add tqdm in the train method, don't print too much here
    # To-Do, fix warning
    train(train_loader, model, criterion, optimizer, epoch, normalizer)

    # evaluate on validation set
    mae_error = validate(val_loader, model, criterion, normalizer)

    if mae_error != mae_error:
        print('Exit due to NaN')
        sys.exit(1)

    scheduler.step()

    # remember the best mae_eror and save checkpoint
    is_best = mae_error < best_mae_error
    best_mae_error = min(mae_error, best_mae_error)

    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_mae_error': best_mae_error,
        'optimizer': optimizer.state_dict(),
        'normalizer': normalizer.state_dict(),
    }, is_best)


/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Epoch: [0][0/67]	Time 4.174 (4.174)	Data 4.149 (4.149)	Loss 0.2806 (0.2806)	MAE 33.661 (33.661)
Epoch: [0][10/67]	Time 0.173 (0.506)	Data 0.144 (0.481)	Loss 0.2260 (0.4284)	MAE 27.904 (31.323)
Epoch: [0][20/67]	Time 0.199 (0.329)	Data 0.168 (0.304)	Loss 0.9276 (0.4242)	MAE 40.531 (31.370)
Epoch: [0][30/67]	Time 0.099 (0.266)	Data 0.076 (0.240)	Loss 0.2496 (0.4043)	MAE 28.405 (30.997)
Epoch: [0][40/67]	Time 0.131 (0.239)	Data 0.103 (0.213)	Loss 0.0990 (0.4683)	MAE 21.543 (31.908)
Epoch: [0][50/67]	Time 0.173 (0.222)	Data 0.140 (0.196)	Loss 0.7011 (0.4656)	MAE 49.489 (32.808)
Epoch: [0][60/67]	Time 0.185 (0.217)	Data 0.159 (0.190)	Loss 0.6097 (0.5658)	MAE 55.410 (34.588)


/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Test: [0/9]	Time 3.988 (3.988)	Loss 0.2735 (0.2735)	MAE 34.290 (34.290)


/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


...

Epoch: [14][0/67]	Time 4.046 (4.046)	Data 4.021 (4.021)	Loss 0.1527 (0.1527)	MAE 28.092 (28.092)
Epoch: [14][10/67]	Time 0.107 (0.499)	Data 0.081 (0.472)	Loss 0.2020 (0.2569)	MAE 31.608 (28.263)
Epoch: [14][20/67]	Time 0.205 (0.340)	Data 0.170 (0.311)	Loss 0.1723 (0.2780)	MAE 22.488 (29.299)
Epoch: [14][30/67]	Time 0.117 (0.287)	Data 0.097 (0.257)	Loss 0.1051 (0.2559)	MAE 18.838 (26.860)
Epoch: [14][40/67]	Time 0.197 (0.256)	Data 0.163 (0.226)	Loss 0.1952 (0.3270)	MAE 29.434 (27.938)
Epoch: [14][50/67]	Time 0.186 (0.237)	Data 0.154 (0.206)	Loss 0.1825 (0.3276)	MAE 29.538 (28.590)
Epoch: [14][60/67]	Time 0.126 (0.225)	Data 0.104 (0.195)	Loss 0.2848 (0.3781)	MAE 33.501 (30.566)


/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Test: [0/9]	Time 4.526 (4.526)	Loss 0.3072 (0.3072)	MAE 37.472 (37.472)


/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


 * MAE 42.685


In [12]:
# test best model
print('---------Evaluate Model on Test Set---------------')
best_checkpoint = torch.load('model_best.pth.tar')
model.load_state_dict(best_checkpoint['state_dict'])
validate(test_loader, model, criterion, normalizer, test=True)

---------Evaluate Model on Test Set---------------


/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/XChen9/opt/anaconda3/envs/cs282_env/lib/python3.10/site-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Test: [0/9]	Time 4.027 (4.027)	Loss 0.7605 (0.7605)	MAE 48.910 (48.910)
 ** MAE 54.624


tensor(54.6244)

In [ ]:
# hw task, visualize the training loss